# Import Libraries

In [ ]:
from abc import ABC, abstractmethod
from bs4 import BeautifulSoup

import regex as re
import json
import os
from datetime import date

import numpy as np
import pandas as pd

# Define Team Class

In [ ]:
class Team:
    def __init__(self, rank, name, institution, solved, penalty, first_solve_count):
        self.rank = rank
        self.name = name
        self.institution = institution.upper()
        self.solved = solved
        self.penalty = penalty
        self.first_solve_count = first_solve_count

    def __lt__(self, other):
        if not isinstance(other, Team): return NotImplemented
        return self.rank < other.rank

    def __repr__(self):
        return json.dumps(self.__dict__)

# Define ContestParser Class

In [ ]:
class ContestParser(ABC):
    _parsers = {}

    def parse(self, filepaths):
        team_list = []
        for filepath in filepaths:
            team_list.extend(self.parseFile(filepath))
        return team_list

    @abstractmethod
    def parseFile(self, filepath):
        """
        Parses a single HTML file containing contest standings and returns a list of Team objects.

        Args:
            filepath (str): Path to the HTML file containing the contest data.

        Returns:
            list[Team]: A list of Team objects.
        """
        pass

    @classmethod
    def get_parser(cls, key):
        """
        Returns a parser instance for the given key.

        Args:
            key (str): The key identifying the parser.

        Returns:
            ContestParser: The parser instance corresponding to the key.
        """
        key = key.upper()
        if key not in cls._parsers: raise ValueError(f"No parser found for key: {key}")

        return cls._parsers[key]

    @classmethod
    def register_parser(cls, key, parser_instance):
        """
        Registers a parser instance with the specified key.

        Args:
            key (str): The key to associate with the parser.
            parser_instance (ContestParser): The parser instance to register.
        """
        if key.upper() in cls._parsers: raise ValueError(f"Parser for key '{key}' is already registered.")

        cls._parsers[key.upper()] = parser_instance

## Define Implementations of ContestParser Class

In [ ]:
class TophParser(ContestParser):
    def parseFile(self, filepath):
        with open(filepath, "r", encoding="utf-8") as file:
            contest_html = file.read()

        soup = BeautifulSoup(contest_html, "html.parser")
        table = soup.find("table")
        if not table: raise ValueError("No table found in the HTML document.")

        rows = table.find_all("tr")
        team_list = []

        for row in rows[1:]:
            cells = row.find_all("td")
            if len(cells) < 3: continue

            try:
                rank = int(cells[0].get_text(strip=True))

                team_name = cells[1].contents[0].strip()
                institution_div = cells[1].find("div", class_="adjunct")
                institution = institution_div.get_text(strip=True) if institution_div else ""

                solve_count = int(cells[2].find("strong").get_text(strip=True))
                penalty_text = cells[2].find("div", class_="adjunct").get("data-tippy-content")
                penalty = int(re.search(r"Penalty: (\d+)", penalty_text).group(1))

                first_solve_count = sum(
                    1 for cell in cells[3:]
                    if cell.find("img", class_="icon green") and
                    cell.find("img", class_="icon green").get("data-tippy-content") == "First to Solve"
                )

                team = Team(
                    name=team_name,
                    institution=institution,
                    rank=rank,
                    solved=solve_count,
                    penalty=penalty,
                    first_solve_count=first_solve_count,
                )
                team_list.append(team)
            except Exception as e:
                print(f"Error processing row: {row}")
                print(e)

        return team_list

ContestParser.register_parser("toph", TophParser())

In [ ]:
class BAPSparser(ContestParser):
    def parseFile(self, filepath):
        with open(filepath, "r", encoding="utf-8") as file:
            contest_html = file.read()

        soup = BeautifulSoup(contest_html, "html.parser")
        table = soup.find("table")
        if not table: raise ValueError("No table found in the HTML document.")

        rows = table.find_all("tr")
        team_list = []

        for row in rows[1:]:
            cells = row.find_all("td")
            if len(cells) < 3: continue

            try:
                rank = int(cells[0].get_text(strip=True))
                team_name = cells[1].find("strong").get_text(strip=True)
                institution_div = cells[1].find("div")
                institution = institution_div.get_text(strip=True) if institution_div else ""
                solve_count_text = cells[2].get_text(strip=True)
                solve_count = int(re.search(r"(\d+)", solve_count_text).group(1))
                penalty = int(re.search(r"\((\d+)\)", solve_count_text).group(1))
                first_solve_count = sum(
                    1 for cell in cells[3:]
                    if cell.find("div", style=re.compile(r"animation:.*shine.*"))
                )

                team = Team(
                    name=team_name,
                    institution=institution,
                    rank=rank,
                    solved=solve_count,
                    penalty=penalty,
                    first_solve_count=first_solve_count,
                )
                team_list.append(team)
            except Exception as e:
                print(f"Error processing row: {row}")
                print(e)

        return team_list

ContestParser.register_parser("baps", BAPSparser())

# Define Contest Class

In [ ]:
class Contest:
    def __init__(self, name, filepaths, parser, date_string):
        if not isinstance(parser, ContestParser):
            raise TypeError("parser must be an instance of ContestParser")

        self.name = name.upper()
        self.team_list = parser.parse(filepaths)

        self.max_solved = 0
        self.institution_map = {}
        self.date = date.fromisoformat(date_string)

        for team in self.team_list:
            self.max_solved = max(self.max_solved, team.solved)

            if team.institution not in self.institution_map: self.institution_map[team.institution] = []
            self.institution_map[team.institution].append(team)

    def __lt__(self, other):
        if not isinstance(other, Contest): return NotImplemented
        return self.date < other.date

    def __repr__(self):
        return json.dumps({
            "name": self.name,
            "date": self.date.isoformat(),
            "max_solved": self.max_solved,
            "team_list": [team.__dict__ for team in self.team_list],
            "institution_map": {k: [team.__dict__ for team in v] for k, v in self.institution_map.items()}
        }, indent=4)

# Define Institution Class

In [ ]:
class Institution:
    def __init__(self, name, alt_names=[]):
        self.name = name.upper()
        self.alt_names = [alt_name.upper() for alt_name in alt_names]
        self.contest_map = {}

    def add_contest(self, contest):
        name = self.name
        alt_names = self.alt_names
        institution_map = contest.institution_map
        contest_name = contest.name

        contest_team_list = []

        if name in institution_map:
            contest_team_list.extend(institution_map[name])

        for alt_name in alt_names:
            if alt_name in institution_map:
                contest_team_list.extend(institution_map[alt_name])

        if len(contest_team_list) > 0:
            self.contest_map[contest_name] = sorted(contest_team_list)

    def get_contest_teams(self, contest_name):
        if contest_name in self.contest_map:
            return self.contest_map[contest_name]
        return None

    def __repr__(self):
        return json.dumps({
            "name": self.name,
            "alt_names": self.alt_names,
            "contest_map": {
                contest_name: [team.__dict__ for team in teams]
                for contest_name, teams in self.contest_map.items()
            }
        }, indent=4)

# Load Data

In [ ]:
def load_contests_from_json(file_path):
    contest_dir = "./input/grading/contest_files/"
    contests = {}

    with open(file_path, "r") as file:
        data = json.load(file)

        for contest in data["contests"]:
            name = contest["name"]
            filepaths = [contest_dir + filename for filename in contest["filenames"]]
            parser = ContestParser.get_parser(contest["parser"])
            date_string = contest["date"]
            contests[name] = Contest(name, filepaths, parser, date_string)

    return contests

def load_institutions_from_json(file_path):
    institutions = []

    with open(file_path, "r") as file:
        data = json.load(file)

        for institution_data in data["institutions"]:
            name = institution_data["name"]
            alt_names = institution_data.get("alt_names", [])
            institutions.append(Institution(name, alt_names))

    return institutions

In [ ]:
contests_file_path = "./input/grading/contests.json"
contests = load_contests_from_json(contests_file_path)

institutions_file_path = "./input/grading/institutions.json"
institutions = load_institutions_from_json(institutions_file_path)

In [ ]:
contest_list = sorted(contests.values(), reverse=True)
contest_name_list = [contest.name for contest in contest_list]

In [ ]:
for institution in institutions:
    for contest in contests.values():
        institution.add_contest(contest)

In [ ]:
credits_file_path = "./input/grading/credits.json"
with open(credits_file_path, "r") as file: credits_map = json.load(file)

# Define GradeCalculator Class

In [ ]:
class GradeCalculator:
    def get_grade_point(self, institution, contest_name):
        contest_teams = institution.get_contest_teams(contest_name)
        if not contest_teams: return None

        contest = contests[contest_name]

        best4 = sorted(contest_teams)[:4]
        team_grades = []

        rank_decay_rate = 0.02
        for team in best4:
            grade = 4 * (1 - rank_decay_rate) ** (team.rank - 1) * (team.solved / contest.max_solved) ** (1/2)

            team_grades.append(grade)

        k = 3.14159
        lk_norm = np.mean(np.array(team_grades) ** k) ** (1 / k)
        return lk_norm
    
    def get_cgpa(self, institution_list, contest_list, credits_map):
        contest_name_list = [c.name for c in contest_list]

        marksheet = pd.DataFrame(
            index=[inst.name for inst in institution_list],
            columns=["CGPA"] + [contest_name + " GP" for contest_name in contest_name_list]
        )

        marksheet["GP L2"] = 0.0
        marksheet["Credit L2"] = 0.0

        latest_contest_date = contest_list[0].date
        time_decay_rate = 0.25
        decay_period_unit = 91.3125 # 3 months

        for contest in contest_list:
            contest_name = contest.name
            credit = credits_map[contest_name]

            period = (latest_contest_date - contest.date).days // decay_period_unit
            weight = credit * ((1 - time_decay_rate) ** period)

            for institution in institution_list:
                gp = self.get_grade_point(institution, contest_name)
                marksheet.at[institution.name, contest_name + " GP"] = gp

                if gp is not None:
                    marksheet.at[institution.name, "GP L2"] += (gp * weight) ** 2
                    marksheet.at[institution.name, "Credit L2"] += weight ** 2

        marksheet.loc[marksheet["Credit L2"] > 0, "CGPA"] = (marksheet["GP L2"] / marksheet["Credit L2"]) ** (1 / 2)
        marksheet.loc[marksheet["Credit L2"] == 0, "CGPA"] = 0

        marksheet.drop(columns=["GP L2", "Credit L2"], inplace=True)
        marksheet = marksheet.infer_objects(copy=False)

        return marksheet

# Perform Calculations

In [ ]:
calculator = GradeCalculator()

marksheet = calculator.get_cgpa(institutions, contest_list, credits_map)
ranked_df = marksheet.sort_values(by="CGPA", ascending=False)
display(ranked_df)

# Export Results

In [ ]:
os.makedirs('result', exist_ok=True)

In [ ]:
with open("./result/contests.json", "w") as file:
    json.dump(json.loads(contest_list.__repr__()), file, indent=4)

In [ ]:
with open("./result/institutions.json", "w") as file:
    json.dump(json.loads(institutions.__repr__()), file, indent=4)

In [ ]:
marksheet_df = ranked_df.reset_index()
marksheet_df.rename(columns={"index": "Institution"}, inplace=True)

marksheet_df.to_csv("./result/marksheet.csv", index=False)